In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls "/content/drive/My Drive/Project"
!pip install -U scikit-learn

Mounted at /content/drive
aftertrain.csv	 logregmodel.pkl  test.csv   train.csv
corona_fake.csv  submit.csv	  tfidf.pkl
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.1)


In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess(df):
  ps= PorterStemmer()
  stopWords = set(stopwords.words('english'))
  train['pro']=' '
  for i in range(len(df['full'])):
    words = word_tokenize(df['full'][i])
    stem=[]
    for w in words:
      if w not in stopWords:
        stem.append(ps.stem(w))
    df.loc[i,'pro']=' '.join(stem)

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Project/train.csv')
test = pd.read_csv('/content/drive/My Drive/Project/test.csv')
corona=pd.read_csv('/content/drive/My Drive/Project/corona_fake.csv')

In [ ]:
print(train['label'])
test['label']  = '2'
test['label']

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


0       2
1       2
2       2
3       2
4       2
       ..
5195    2
5196    2
5197    2
5198    2
5199    2
Name: label, Length: 5200, dtype: object

In [ ]:
test=test.fillna(' ')
train=train.fillna(' ')
corona=corona.fillna(' ')
test['full']=test['title']+' '+test['author']+test['text']
train['full']=train['title']+' '+train['author']+train['text']
corona['full']=corona['text']

In [ ]:
for i in range(len(corona['label'])):
  if corona['label'][i] == "Fake":
    corona['label'][i] = np.int64(1)
  else:
    corona['label'][i] = np.int64(0)

corona_label=np.array(corona['label'],dtype='int')
train_label=np.array(train['label'])

In [ ]:
preprocess(test)
preprocess(corona)
preprocess(train)

In [ ]:
train_list = list(train['pro'].values)
print(len(train_list))
corona_list = list(corona['pro'].values)
final_list= train_list + corona_list
print(len(final_list))

trainy_list = list(np.array(train['label']))
print(len(trainy_list))
coronay_list = list(np.array(corona['label'],dtype='int'))
finaly_list= trainy_list + coronay_list
print(len(finaly_list))

20800
21964
20800
21964


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(train['pro'].values, train_label, random_state=0) 
X_train, X_test, y_train, y_test = train_test_split(final_list, finaly_list, random_state=0)

In [ ]:
#tfidf_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3),max_df=0.5,smooth_idf=True,norm='l2')
tfidf_vectorizer = TfidfVectorizer(analyzer='word',lowercase=True, max_df=0.5, max_features=None, ngram_range=(1, 2), norm='l2',use_idf=True)
#tfidf_train = tfidf_vectorizer.fit(train['pro'].values)
tfidf_train = tfidf_vectorizer.fit(final_list)
#tfidf_train_ans = train['label'].values
tfidf_train_ans = finaly_list
tfidf_train = tfidf_vectorizer.transform(final_list)
corona_train= tfidf_vectorizer.transform(corona['pro'].values)
#tfidf_test = tfidf_vectorizer.fit_transform(test['full'].values)

In [ ]:
X_train=tfidf_vectorizer.transform(X_train)
X_test=tfidf_vectorizer.transform(X_test)

In [ ]:
modelNB = MultinomialNB()   #NaiveBayes
modelNB.fit(X_train, y_train)
#print(modelNB.score(corona_train,corona_label))
print('Accuracy of NaiveBayes classifier on training set: {:.2f}'.format(modelNB.score(X_train, y_train)))
print('Accuracy of NaiveBayes classifier on test set: {:.2f}'.format(modelNB.score(X_test, y_test)))

Accuracy of NaiveBayes classifier on training set: 0.93
Accuracy of NaiveBayes classifier on test set: 0.82


In [ ]:
#logreg = LogisticRegression(max_iter=200,random_state=0)
logreg=LogisticRegression(C=25, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, l1_ratio=None, max_iter=1000,multi_class='auto', n_jobs=None, penalty='l2',random_state=0, solver='lbfgs', tol=0.0001, verbose=0,warm_start=False)
logreg.fit(X_train, y_train)
print('Accuracy of LogisticRegress classifier on training set: {:.3f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of LogisticRegress classifier on test set: {:.3f}'.format(logreg.score(X_test, y_test)))

Accuracy of LogisticRegress classifier on training set: 0.999
Accuracy of LogisticRegress classifier on test set: 0.963


In [ ]:
print('Accuracy of LogisticRegress classifier on COVID dataset : {:.3f}'.format(logreg.score(corona_train, corona_label)))

Accuracy of LogisticRegress classifier on COVID dataset : 0.930


In [ ]:
submit = pd.read_csv('/content/drive/My Drive/Project/submit.csv')
tfdif2 = tfidf_vectorizer.transform(test['pro'].values)
predictions = logreg.predict(tfdif2)
pred = pd.DataFrame(predictions,columns=['label'])
pred['id']=test['id']
pred.groupby('label').count()

,id
label,
0,2645
1,2555


In [ ]:
accuracy=(sum(pred['label'] == submit['label']))/(len(pred['label']))
print(accuracy)

0.6357692307692308


In [ ]:
article=input("Enter article")
ps=PorterStemmer()
stopWords = set(stopwords.words('english'))
words = word_tokenize(article)
stem=[]
for w in words:
  if w not in stopWords:
    stem.append(ps.stem(w))
finalart=' '.join(stem)
final=[]
final.append(finalart)
print(final)
test1=tfidf_vectorizer.transform(final)
if logreg.predict(test1)==0:
  print("News is reliable")
else:
    print("News is unreliable")
#print(logreg.predict(test1))




Enter articleas
['']
News is unreliable


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier()

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
print('Accuracy of XGB classifier on training set: {:.2f}'.format(model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))

Accuracy of XGB classifier on training set: 0.96
Accuracy of XGB classifier on test set: 0.96


In [ ]:
print('Accuracy of XGB classifier  on COVID dataset : {:.3f}'.format(model.score(corona_train, corona_label)))

Accuracy of XGB classifier  on COVID dataset : 0.704


In [ ]:
import pickle
pickle.dump(tfidf_vectorizer,open('/content/drive/My Drive/Project/tfidf.pkl','wb'))

In [ ]:
import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(logreg, '/content/drive/My Drive/Project/logregmodel.pkl')


['/content/drive/My Drive/Project/logregmodel.pkl']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(smooth_idf=True)),
    ('clf', LogisticRegression(max_iter=1000)),
])
parameters = {
    'clf__C':[10,25,45,50,55,60],
    'clf__tol':[0.0001,0.001,0.01,0.1,1]
}

grid_search_tune = GridSearchCV(pipeline, parameters)
grid_search_tune.fit(final_list, finaly_list)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

Best parameters set:
[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression(C=45, max_iter=1000, tol=1))]
